In [86]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI =  "http://127.0.0.1:8080"


In [87]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
#client.create_experiment(name="my-cool-experiment")


In [88]:
list_experiment = client.get_experiment(1)
for exp in list_experiment:
    print(exp)

('artifact_location', 'mlflow-artifacts:/1')
('creation_time', 1716324856379)
('experiment_id', '1')
('last_update_time', 1716324856379)
('lifecycle_stage', 'active')
('name', 'nyc-taxi-experiment')
('tags', {})


In [89]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 5.4",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [90]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 733d77119f9c44f28f451ee7ad09ccd7, rmse: 5.1237
run id: cc28bd5925164702a579fee440670ca4, rmse: 5.1265
run id: f5e08c093e3c45dfac8cd95152cbf3ad, rmse: 5.1346
run id: 54fa4a7dd2b8490598a29761e73c1d3d, rmse: 5.1396
run id: c5f436ce49024072b6699fcb1c4c0d93, rmse: 5.1403


### Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the experiment nyc-taxi-regressor.

Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.

Transition the version 4 to "Staging" and adding annotations to it.

In [91]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [92]:
run_id = "733d77119f9c44f28f451ee7ad09ccd7"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2024/05/22 22:21:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 5
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716416498447, current_stage='None', description='', last_updated_timestamp=1716416498447, name='nyc-taxi-regressor', run_id='733d77119f9c44f28f451ee7ad09ccd7', run_link='', source='mlflow-artifacts:/1/733d77119f9c44f28f451ee7ad09ccd7/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

In [93]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 5, stage: None
version: 3, stage: Staging
version: 4, stage: Production


/tmp/ipykernel_32530/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [94]:
model_name = 'nyc-taxi-regressor'
client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version:{version.version}, stage: {version.current_stage}")

version:5, stage: None
version:3, stage: Staging
version:4, stage: Production


/tmp/ipykernel_32530/3921306844.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


In [95]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_32530/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716413584334, current_stage='Staging', description='', last_updated_timestamp=1716416498682, name='nyc-taxi-regressor', run_id='733d77119f9c44f28f451ee7ad09ccd7', run_link='', source='mlflow-artifacts:/1/733d77119f9c44f28f451ee7ad09ccd7/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

### Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [96]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [97]:
def read_dataframe(filename):
    print(f"Reading file: {filename}")
    df = pd.read_parquet(filename)
    print("File read successfully")
    
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

    # Filter for trip durations between 1 and 60 minutes
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    # Convert categorical columns to string type
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [98]:
df = read_dataframe('/workspaces/experiment-tracking-mlflow-zoomcamp/data/green_tripdata_2024-01.parquet')

Reading file: /workspaces/experiment-tracking-mlflow-zoomcamp/data/green_tripdata_2024-01.parquet
File read successfully


In [99]:
import pickle

with open("/workspaces/experiment-tracking-mlflow-zoomcamp/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [100]:
X_test = preprocess(df, dv)

In [101]:
target = "duration"
y_test = df[target].values

In [102]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)


/tmp/ipykernel_32530/1118029502.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716415025152, current_stage='Production', description='', last_updated_timestamp=1716416499582, name='nyc-taxi-regressor', run_id='733d77119f9c44f28f451ee7ad09ccd7', run_link='', source='mlflow-artifacts:/1/733d77119f9c44f28f451ee7ad09ccd7/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>